In [1]:
import pickle
import time
import random

class MinConflictSolver:
    def __init__(self, nam_file):
        with open(nam_file, 'rb') as f:
            # self.codes_int is the NAM where 0 = neighbor, 1 = conflict
            self.codes_int = pickle.load(f)
        self.num_nodes = len(self.codes_int) - 1

    def are_neighbors(self, u: int, v: int) -> bool:
        if u == v:
            return False
        # If the bit at position v is 0, they ARE neighbors
        return ((self.codes_int[u] >> v) & 1) == 0

    def solve_mccp(self, v_self):
        # 1. INITIALIZATION: Build S with v_self and all its neighbors
        S = [v_self]
        for i in range(1, self.num_nodes + 1):
            if i != v_self and self.are_neighbors(v_self, i):
                S.append(i)
        
        k = 0 # Iteration counter for Landau Complexity O(n*k)
        
        while True:
            # 2. CONFLICT IDENTIFICATION
            # Count how many members in S are NOT neighbors with each node u
            conflicts = {}
            for u in S:
                # We only need to check neighbors for nodes other than v_self
                # because v_self is guaranteed to be compatible with everyone in S
                count = 0
                for other in S:
                    if u != other and not self.are_neighbors(u, other):
                        count += 1
                conflicts[u] = count
            
            # Find the vertex with the most conflicts (excluding v_self)
            max_c = -1
            victim = -1
            for u in S:
                if u == v_self: continue
                if conflicts[u] > max_c:
                    max_c = conflicts[u]
                    victim = u
            
            # 3. TERMINATION: If no neighbor has conflicts, we have a clique
            if max_c <= 0:
                break
                
            # 4. EJECTION: Remove the "Victim" (Min-Conflict move)
            S.remove(victim)
            k += 1
            
        return len(S), k, S

# --- Execution ---
solver = MinConflictSolver('C125-9_nam.data')
v_test = random.randint(1, 125)

start = time.time()
size, k_val, final_clique = solver.solve_mccp(v_test)
duration = time.time() - start

print(f"Agent {v_test} result on C125.9:")
print(f"Max Clique Size: {size} (Target: 34)")
print(f"Landau k value: {k_val}")
print(f"Time: {duration:.4f}s")

Agent 80 result on C125.9:
Max Clique Size: 32 (Target: 34)
Landau k value: 86
Time: 0.3173s


In [2]:
import pickle
import time
import random

class DistributedMCH:
    def __init__(self, nam_file):
        with open(nam_file, 'rb') as f:
            # self.codes_int: 0 = neighbor (edge), 1 = non-neighbor (conflict)
            self.codes_int = pickle.load(f)
        self.num_nodes = len(self.codes_int) - 1

    def are_neighbors(self, u: int, v: int) -> bool:
        if u == v: return False
        return ((self.codes_int[u] >> v) & 1) == 0

    def validate_clique(self, clique):
        """Formal TCS Validation: Check if the set is a complete subgraph."""
        for i in range(len(clique)):
            for j in range(i + 1, len(clique)):
                if not self.are_neighbors(clique[i], clique[j]):
                    return False, (clique[i], clique[j])
        return True, None

    def solve_mccp_stochastic(self, v_self, max_restarts=100):
        best_clique = []
        best_size = 0
        total_k_for_best = 0
        
        print(f"Starting Distributed MCH for Agent {v_self} (Target: 34)...")
        
        for trial in range(1, max_restarts + 1):
            # 1. Initialization (Include v_self + all its neighbors)
            neighbors = [i for i in range(1, self.num_nodes + 1) 
                         if i != v_self and self.are_neighbors(v_self, i)]
            
            # Stochastic shuffle to explore different areas of the dense graph
            random.shuffle(neighbors)
            S = [v_self] + neighbors
            
            k = 0
            while True:
                # 2. Conflict Identification
                conflicts = {}
                for u in S:
                    # Logic: count non-adjacencies in current set S
                    c_count = sum(1 for other in S if u != other and not self.are_neighbors(u, other))
                    conflicts[u] = c_count
                
                # 3. Victim Selection (Excluding v_self)
                # Filter candidates who have the maximum conflict count
                potential_victims = [u for u in S if u != v_self]
                if not potential_victims: break
                
                max_c = max(conflicts[u] for u in potential_victims)
                
                if max_c == 0: # Convergence: S is a clique
                    break
                
                # If ties exist, pick randomly to diversify the search path
                victims_pool = [u for u in potential_victims if conflicts[u] == max_c]
                victim = random.choice(victims_pool)
                
                # 4. Ejection
                S.remove(victim)
                k += 1
            
            # Check if this restart found a better group
            if len(S) > best_size:
                best_size = len(S)
                best_clique = S
                total_k_for_best = k
                
            # Exit early if target is reached
            if best_size >= 34:
                return best_clique, best_size, total_k_for_best, trial

        return best_clique, best_size, total_k_for_best, max_restarts

# --- Execution and Validation ---
solver = DistributedMCH('C125-9_nam.data')

# We pick a node to act as the 'Centric Agent'
agent_id = random.randint(1, 125)

start_time = time.time()
clique, size, k, trials = solver.solve_mccp_stochastic(agent_id)
execution_time = time.time() - start_time

# Validation Step
is_valid, error_pair = solver.validate_clique(clique)

print("-" * 30)
print(f"Agent ID: {agent_id}")
print(f"Max Clique Found: {size}")
print(f"Landau Complexity (k): {k}")
print(f"Total Restarts: {trials}")
print(f"Execution Time: {execution_time:.4f} seconds")
print(f"Validation (Is it a Clique?): {is_valid}")
if not is_valid:
    print(f"Error: {error_pair[0]} and {error_pair[1]} are not compatible.")
print(f"Clique Members: {sorted(clique)}")
print("-" * 30)

Starting Distributed MCH for Agent 48 (Target: 34)...
------------------------------
Agent ID: 48
Max Clique Found: 33
Landau Complexity (k): 81
Total Restarts: 100
Execution Time: 22.8899 seconds
Validation (Is it a Clique?): True
Clique Members: [2, 5, 7, 9, 11, 19, 25, 29, 31, 34, 40, 44, 48, 49, 54, 65, 67, 68, 70, 71, 77, 79, 80, 82, 93, 96, 98, 104, 110, 114, 117, 122, 125]
------------------------------


In [9]:
import pickle
import time
import random

class MinConflictSolver:
    def __init__(self, nam_file):
        with open(nam_file, 'rb') as f:
            # self.codes_int: Upper triangular bitmask
            # 0 = neighbor, 1 = conflict
            self.codes_int = pickle.load(f)
        self.num_nodes = len(self.codes_int) - 1

    def are_neighbors(self, u: int, v: int) -> bool:
        if u == v:
            return False
        # Upper Triangular Logic: ensure u is the smaller index
        if u > v:
            u, v = v, u
        return ((self.codes_int[u] >> v) & 1) == 0

    def solve_mccp(self, v_self):
        # 1. INITIALIZATION: Build S with v_self and its neighborhood
        # Using your specific neighbor logic
        S = [v_self]
        for i in range(1, self.num_nodes + 1):
            if i != v_self and self.are_neighbors(v_self, i):
                S.append(i)
        
        k = 0 # Number of ejections
        
        while True:
            # 2. CONFLICT IDENTIFICATION
            conflicts = {}
            for u in S:
                count = 0
                for other in S:
                    if u != other and not self.are_neighbors(u, other):
                        count += 1
                conflicts[u] = count
            
            # 3. VICTIM SELECTION
            # Find vertex with most conflicts (excluding the centric agent v_self)
            max_c = -1
            victim = -1
            for u in S:
                if u == v_self: continue 
                if conflicts[u] > max_c:
                    max_c = conflicts[u]
                    victim = u
            
            # 4. TERMINATION: If max conflicts is 0, S is a clique
            if max_c <= 0:
                break
                
            # 5. EJECTION
            S.remove(victim)
            k += 1
            
        return len(S), k, S

# --- Execution for Keller 3 ---
# Keller 3 benchmark: n=31, omega=5
solver = MinConflictSolver('keller_codes_d3.data')
v_test = random.randint(1, 31)

start = time.time()
size, k_val, final_clique = solver.solve_mccp(v_test)
duration = time.time() - start

print("-" * 40)
print(f"Agent {v_test} Results (Keller 3)")
print("-" * 40)
print(f"Max Clique Size Found: {size} (Target: 5)")
print(f"Landau Iterations (k): {k_val}")
print(f"Convergence Time:      {duration:.6f}s")
print(f"Final Clique Members:  {sorted(final_clique)}")
print("-" * 40)

----------------------------------------
Agent 25 Results (Keller 3)
----------------------------------------
Max Clique Size Found: 4 (Target: 5)
Landau Iterations (k): 30
Convergence Time:      0.012863s
Final Clique Members:  [23, 25, 54, 63]
----------------------------------------


In [10]:
import pickle
import time
import random

class MinConflictSolver:
    def __init__(self, nam_file):
        with open(nam_file, 'rb') as f:
            self.codes_int = pickle.load(f)
        self.num_nodes = len(self.codes_int) - 1

    def are_neighbors(self, u: int, v: int) -> bool:
        if u == v: return False
        if u > v: u, v = v, u
        return ((self.codes_int[u] >> v) & 1) == 0

    def solve_mccp(self, v_self, target=5, max_restarts=50):
        best_clique = []
        best_size = 0
        total_k = 0

        for trial in range(1, max_restarts + 1):
            # 1. INITIALIZATION
            neighbors = [i for i in range(1, self.num_nodes + 1) 
                         if i != v_self and self.are_neighbors(v_self, i)]
            
            # Shuffling the initial neighborhood provides different starting points
            random.shuffle(neighbors)
            S = [v_self] + neighbors
            
            k = 0 
            while True:
                # 2. CONFLICT IDENTIFICATION
                conflicts = {u: sum(1 for o in S if u != o and not self.are_neighbors(u, o)) for u in S}
                
                # 3. VICTIM SELECTION (Stochastic Tie-Breaking)
                potential_victims = [u for u in S if u != v_self]
                if not potential_victims: break
                
                max_c = max(conflicts[u] for u in potential_victims)
                if max_c == 0: break
                
                # SHUFFLING THE POOL: Pick any node that has the maximum conflict count
                victims_pool = [u for u in potential_victims if conflicts[u] == max_c]
                victim = random.choice(victims_pool)
                
                S.remove(victim)
                k += 1
            
            if len(S) > best_size:
                best_size = len(S)
                best_clique = S
                total_k = k
            
            # If we hit the Keller 3 target, we stop
            if best_size >= target:
                return best_size, total_k, best_clique, trial

        return best_size, total_k, best_clique, max_restarts

# --- Execution ---
solver = MinConflictSolver('keller_codes_d3.data')
v_test = random.randint(1, 31)

start = time.time()
size, k_val, final_clique, trials = solver.solve_mccp(v_test)
duration = time.time() - start

print("-" * 40)
print(f"Agent {v_test} Results (Keller 3)")
print("-" * 40)
print(f"Max Clique Size Found: {size} (Target: 5)")
print(f"Total Trials Required: {trials}")
print(f"Landau Iterations (k): {k_val}")
print(f"Time:                  {duration:.6f}s")
print(f"Final Clique Members:  {sorted(final_clique)}")
print("-" * 40)

----------------------------------------
Agent 20 Results (Keller 3)
----------------------------------------
Max Clique Size Found: 5 (Target: 5)
Total Trials Required: 4
Landau Iterations (k): 30
Time:                  0.041469s
Final Clique Members:  [12, 20, 26, 53, 62]
----------------------------------------


In [11]:
import pickle
import time
import random
import statistics

class MinConflictSolver:
    def __init__(self, nam_file):
        with open(nam_file, 'rb') as f:
            self.codes_int = pickle.load(f)
        self.num_nodes = len(self.codes_int) - 1

    def are_neighbors(self, u: int, v: int) -> bool:
        if u == v: return False
        if u > v: u, v = v, u
        return ((self.codes_int[u] >> v) & 1) == 0

    def solve_mccp(self, v_self, target=5, max_trials=50):
        # 1. INITIALIZATION
        neighbors = [i for i in range(1, self.num_nodes + 1) 
                     if i != v_self and self.are_neighbors(v_self, i)]
        random.shuffle(neighbors)
        S = [v_self] + neighbors
        
        k = 0 
        while True:
            conflicts = {u: sum(1 for o in S if u != o and not self.are_neighbors(u, o)) for u in S}
            potential_victims = [u for u in S if u != v_self]
            if not potential_victims: break
            
            max_c = max(conflicts[u] for u in potential_victims)
            if max_c == 0: break
            
            victims_pool = [u for u in potential_victims if conflicts[u] == max_c]
            victim = random.choice(victims_pool)
            S.remove(victim)
            k += 1
        
        return len(S), k, S

def run_batch_test(nam_file, runs_per_agent=10, target_size=5):
    solver = MinConflictSolver(nam_file)
    results_log = []
    
    total_successes = 0
    all_times = []
    all_k = []
    
    print(f"Starting Batch Test: {solver.num_nodes} agents, {runs_per_agent} runs each...")
    
    with open("batch_results.txt", "w") as f:
        f.write(f"Batch Test Results for {nam_file}\n")
        f.write("AgentID | Run | Size | k-Value | Time(s) | Success\n")
        f.write("-" * 50 + "\n")

        for agent in range(1, solver.num_nodes + 1):
            agent_success_count = 0
            for run in range(1, runs_per_agent + 1):
                start = time.time()
                size, k, clique = solver.solve_mccp(agent, target=target_size)
                elapsed = time.time() - start
                
                success = 1 if size >= target_size else 0
                total_successes += success
                agent_success_count += success
                all_times.append(elapsed)
                all_k.append(k)
                
                # Log to file
                f.write(f"{agent:7} | {run:3} | {size:4} | {k:7} | {elapsed:.6f} | {success}\n")
            
            if agent % 10 == 0:
                print(f"Completed Agents 1 to {agent}...")

    # Calculate Statistics
    total_runs = solver.num_nodes * runs_per_agent
    success_rate = (total_successes / total_runs) * 100
    avg_time = statistics.mean(all_times)
    avg_k = statistics.mean(all_k)
    
    print("\n" + "="*30)
    print("FINAL STATISTICS")
    print("="*30)
    print(f"Total Runs:      {total_runs}")
    print(f"Success Rate:    {success_rate:.2f}%")
    print(f"Avg Time:        {avg_time:.6f} s")
    print(f"Avg Ejections(k): {avg_k:.2f}")
    print(f"Results saved to: batch_results.txt")

# --- Run the Batch ---
run_batch_test('keller_codes_d3.data')

Starting Batch Test: 63 agents, 10 runs each...
Completed Agents 1 to 10...
Completed Agents 1 to 20...
Completed Agents 1 to 30...
Completed Agents 1 to 40...
Completed Agents 1 to 50...
Completed Agents 1 to 60...

FINAL STATISTICS
Total Runs:      630
Success Rate:    26.67%
Avg Time:        0.009177 s
Avg Ejections(k): 30.19
Results saved to: batch_results.txt


In [12]:
import pickle

class KellerAnalyzer:
    def __init__(self, nam_file):
        with open(nam_file, 'rb') as f:
            self.codes_int = pickle.load(f)
        self.num_nodes = len(self.codes_int) - 1
        self.max_cliques_5 = []

    def are_neighbors(self, u, v):
        if u == v: return False
        if u > v: u, v = v, u
        return ((self.codes_int[u] >> v) & 1) == 0

    def find_all_5_cliques(self):
        print("Enumerating all cliques of size 5...")
        nodes = list(range(1, self.num_nodes + 1))
        
        def backtrack(current_clique, candidates):
            if len(current_clique) == 5:
                self.max_cliques_5.append(set(current_clique))
                return
            
            for i, v in enumerate(candidates):
                # New candidates must be neighbors with everyone in the current clique
                new_candidates = [n for n in candidates[i+1:] if self.are_neighbors(v, n)]
                backtrack(current_clique + [v], new_candidates)

        backtrack([], nodes)
        
        # Find which nodes belong to at least one 5-clique
        nodes_in_5_cliques = set()
        for c in self.max_cliques_5:
            nodes_in_5_cliques.update(c)
            
        return self.max_cliques_5, nodes_in_5_cliques

# --- Run Enumeration ---
analyzer = KellerAnalyzer('keller_codes_d3.data')
all_5_cliques, valid_agents = analyzer.find_all_5_cliques()

print(f"Total nodes in graph: {analyzer.num_nodes}")
print(f"Total number of size-5 cliques found: {len(all_5_cliques)}")
print(f"Number of agents capable of reaching size 5: {len(valid_agents)}")
print(f"Agents restricted to size 4: {analyzer.num_nodes - len(valid_agents)}")

Enumerating all cliques of size 5...
Total nodes in graph: 63
Total number of size-5 cliques found: 472
Number of agents capable of reaching size 5: 63
Agents restricted to size 4: 0


In [13]:
import pickle
import time
import random
import statistics

class RobustMCHSoler:
    def __init__(self, nam_file):
        with open(nam_file, 'rb') as f:
            self.codes_int = pickle.load(f)
        self.num_nodes = len(self.codes_int) - 1

    def are_neighbors(self, u: int, v: int) -> bool:
        if u == v: return False
        if u > v: u, v = v, u
        return ((self.codes_int[u] >> v) & 1) == 0

    def single_trial(self, v_self):
        """A single greedy pass using Min-Conflict with stochastic tie-breaking."""
        # 1. Initialization
        neighbors = [i for i in range(1, self.num_nodes + 1) 
                     if i != v_self and self.are_neighbors(v_self, i)]
        random.shuffle(neighbors)
        S = [v_self] + neighbors
        
        k = 0
        while True:
            # 2. Conflict Identification
            conflicts = {u: sum(1 for o in S if u != o and not self.are_neighbors(u, o)) for u in S}
            
            # 3. Victim Selection (excluding v_self)
            potential_victims = [u for u in S if u != v_self]
            if not potential_victims: break
            
            max_c = max(conflicts[u] for u in potential_victims)
            if max_c == 0: break # Success: Clique found
            
            # Stochastic Tie-breaking (The 'Memory' of different paths)
            victims_pool = [u for u in potential_victims if conflicts[u] == max_c]
            victim = random.choice(victims_pool)
            
            # 4. Ejection
            S.remove(victim)
            k += 1
            
        return len(S), k, S

    def solve_with_memory(self, v_self, target=5, max_trials=100):
        """Repeats trials until the target clique size is found."""
        start_time = time.time()
        total_ejections = 0
        
        for trial in range(1, max_trials + 1):
            size, k, clique = self.single_trial(v_self)
            total_ejections += k
            
            if size >= target:
                duration = time.time() - start_time
                return True, size, total_ejections, trial, duration, clique
        
        duration = time.time() - start_time
        return False, size, total_ejections, max_trials, duration, clique

# --- Batch Execution Logic ---
def run_final_verification(nam_file):
    solver = RobustMCHSoler(nam_file)
    all_times = []
    all_trials = []
    success_count = 0
    
    print(f"Starting Robust Batch Test: {solver.num_nodes} Agents...")
    
    with open("robust_results.txt", "w") as f:
        f.write("AgentID | Success | Trials | Total_k | Time(s)\n")
        f.write("-" * 50 + "\n")

        for agent in range(1, solver.num_nodes + 1):
            success, size, k, trials, dt, clique = solver.solve_with_memory(agent, target=5)
            
            if success:
                success_count += 1
            
            all_times.append(dt)
            all_trials.append(trials)
            
            f.write(f"{agent:7} | {str(success):7} | {trials:6} | {k:7} | {dt:.6f}\n")
            
            if agent % 10 == 0:
                print(f"Processed {agent}/{solver.num_nodes} agents...")

    # Statistics Summary
    avg_trials = statistics.mean(all_trials)
    avg_time = statistics.mean(all_times)
    final_rate = (success_count / solver.num_nodes) * 100

    print("\n" + "="*35)
    print("ROBUST METHOD FINAL STATISTICS")
    print("="*35)
    print(f"Global Success Rate:  {final_rate:.2f}%")
    print(f"Avg Trials needed:    {avg_trials:.2f}")
    print(f"Avg Total Latency:    {avg_time:.6f} s")
    print(f"Max Trials used:      {max(all_trials)}")
    print(f"Results File:         robust_results.txt")
    print("="*35)

# Run the final check
run_final_verification('keller_codes_d3.data')

Starting Robust Batch Test: 63 Agents...
Processed 10/63 agents...
Processed 20/63 agents...
Processed 30/63 agents...
Processed 40/63 agents...
Processed 50/63 agents...
Processed 60/63 agents...

ROBUST METHOD FINAL STATISTICS
Global Success Rate:  100.00%
Avg Trials needed:    7.56
Avg Total Latency:    0.068219 s
Max Trials used:      74
Results File:         robust_results.txt
